In [13]:
using BirkhoffDecomposition
using JSON3

In [14]:
n = 8
dataset_size = 10000

dataset = Dict([])
for i in range(0,dataset_size)
    X, P, w = BirkhoffDecomposition.randomPermutations(n, num_perm=n)
    entry = Dict([("permutations", Int16.(P)), 
                        ("weights", w), 
                        ("doubly_stochastic_matrix", X),
                        ("n",n)]
            )
   dataset[i] = entry
end
file_name = "dataset_" * string(n) * "_" * string(dataset_size) * ".json"
open(file_name,"a") do io
  JSON3.pretty(io, dataset)
end